## Airborne Tool
Author: Harrison Baker


In [70]:
# package imports
import pandas as pd
import re
import numpy as np


In [ ]:
# adjustment dictionary

periods = ["MS", "Day", "OOH", "Eve", "ES", "Night", "SS", "SA"]

scenarioSettings = ["Scenario Name", "Scenario Description", "Variant Number"]
scenarioSettings.extend(periods)

adjustmentDf = pd.DataFrame(columns=scenarioSettings)
adjustmentDf.loc[0] = ["SCEN1", "a brief description", 1, np.nan, 105, 99, np.nan,  90, 110, 120, 80]
adjustmentDf.loc[1] = ["SCEN2", "a brief description", 1, np.nan, 105, 99, np.nan,  90, 110, 120, 80]
adjustmentDf.loc[2] = ["SCEN3", "a brief description", 1, np.nan, 105, 99, np.nan,  90, 110, 120, 80]

print(adjustmentDf)

  Scenario Name Scenario Description  Variant Number  MS  Day  OOH  Eve  ES  \
0         SCEN1  a brief description               1 NaN  105   99  NaN  90   
1         SCEN2  a brief description               1 NaN  105   99  NaN  90   
2         SCEN3  a brief description               1 NaN  105   99  NaN  90   

   Night   SS  SA  
0    110  120  80  
1    110  120  80  
2    110  120  80  


In [ ]:
def get_period_data(period, resultsDf, adjustmentDf):
    return False

open cadnaA excel file, for each unique NCA_ID take the max value for each variant

In [ ]:
# open cadnaA results as df

# TO DO - get rounding happening then get count tables

cadnaA = pd.read_excel("RawCadnaA.xlsx")
dataColumns = cadnaA.columns[cadnaA.columns.get_loc(1):] # doesn't include NMLs
infoColumns = cadnaA.columns[:cadnaA.columns.get_loc("Day_NML")]

def reduceToMax(df: pd.DataFrame, nca_id = "NCA_ID", nca_tool = "NCA_tool"):
    max_df = df.groupby(nca_id, as_index = False).max().sort_values(by=nca_tool).set_index(nca_id, drop = False)
    max_df.loc[max_df[nca_tool] == "OSR", nca_tool] = max_df[nca_id].str.split('-').str[0]
    return max_df

reduced = reduceToMax(cadnaA)
# print(reduced.head(2))

# print(reduced.head(2))

def calcPeriodLevels(row, corrections: pd.Series, scenarioName, variantId):
    if not variantId in row.index:
        return row
    for periodName, periodValue in corrections.items():
        if periodValue == np.nan:
            continue
        newLevel = row[variantId] + periodValue - 100
        row[f"{scenarioName}_{periodName}"] = newLevel
    return row

def applyAdjustment(df: pd.DataFrame, periods, infoColumns, dataColumns, adjustmentDf, nca_tool = "NCA_tool"):
    
    base = df.copy()
    
    adjusted = base[infoColumns].copy()
    data = base[dataColumns].copy()

    mask = adjusted[nca_tool].str.startswith("NCA")
    base["SS_NML"] = np.where(mask, np.maximum(base["Night_NML"] + 10, 52), 999) # discuss with MT
    base["SA_NML"] = np.where(mask, np.maximum(base["SS_NML"], 65), 999)
    base["OOH_NML"] = np.where(mask, base["Day_NML"] - 5, base["Day_NML"])

    scenarioDict= {
        x: None for x in adjustmentDf["Scenario Name"]
    }

    for id, s in adjustmentDf.iterrows():
        scenarioName = s["Scenario Name"]
        # print(s)
        variantId = s["Variant Number"]
        corrections = s[periods]
        scenarioDf = data.copy()
        scenarioDf = scenarioDf.apply(calcPeriodLevels, scenarioName = scenarioName, variantId = variantId, corrections = corrections, axis = 1)
        scenarioDf = scenarioDf.dropna(axis=1, how='all')
        scenarioDict[scenarioName] = scenarioDf


    for p in periods:
        matches = []
        for name, subDf in scenarioDict.items():
            cols = [c for c in subDf.columns if isinstance(c, str) and '_' in c and c.split('_')[-1] == p]
            if not cols:
                continue

            adjusted[f"{p}_NML"] = base[f"{p}_NML"]

            adjusted = adjusted.join(subDf[cols])

    adjusted.sort_values(by=[nca_tool, "Address"]) # FIX SORT AS COMS ARENT BEING GROUPED
    
    return adjusted, data, scenarioDict

adjusted, data, scenarioDict = applyAdjustment(reduced, periods, infoColumns, dataColumns, adjustmentDf)

adjusted.to_csv('testFullTable.csv', index=False)

print(adjusted.head())

                          NCA_tool                     NCA_ID  \
NCA_ID                                                          
RES-NCA06-00190002497        NCA06      RES-NCA06-00190002497   
OSR_MXU-NCA06-00190007454    NCA06  OSR_MXU-NCA06-00190007454   
OSR_MXU-NCA07-00190007465    NCA07  OSR_MXU-NCA07-00190007465   
OSR_MXU-NCA07-00190007424    NCA07  OSR_MXU-NCA07-00190007424   
OSR_COM-NCA07-00190007340  OSR_COM  OSR_COM-NCA07-00190007340   

                                                           Address  Num_Units  \
NCA_ID                                                                          
RES-NCA06-00190002497            168-170 KENT STREET MILLERS POINT          1   
OSR_MXU-NCA06-00190007454  200  CUMBERLAND STREET, THE ROCKS (SYDN          1   
OSR_MXU-NCA07-00190007465         18A PITT STREET, SYDNEY (SYDNEY)          1   
OSR_MXU-NCA07-00190007424       38  BRIDGE STREET, SYDNEY (SYDNEY)          1   
OSR_COM-NCA07-00190007340   226 - 230 PITT STREET, SYDNEY 